# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [9]:
# Import Python packages 

import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [10]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [20]:
# initiating an empty list of rows that will be generated from each file

full_data_rows_list = [] 

# for every filepath in the file path list 

for f in sorted(file_path_list):
    
# reading csv file 

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        print(f'Adding {f}..........')
        
# extracting each data row one by one and append it       

        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
    
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables

csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


Adding /home/workspace/event_data/2018-11-01-events.csv..........
Adding /home/workspace/event_data/2018-11-02-events.csv..........
Adding /home/workspace/event_data/2018-11-03-events.csv..........
Adding /home/workspace/event_data/2018-11-04-events.csv..........
Adding /home/workspace/event_data/2018-11-05-events.csv..........
Adding /home/workspace/event_data/2018-11-06-events.csv..........
Adding /home/workspace/event_data/2018-11-07-events.csv..........
Adding /home/workspace/event_data/2018-11-08-events.csv..........
Adding /home/workspace/event_data/2018-11-09-events.csv..........
Adding /home/workspace/event_data/2018-11-10-events.csv..........
Adding /home/workspace/event_data/2018-11-11-events.csv..........
Adding /home/workspace/event_data/2018-11-12-events.csv..........
Adding /home/workspace/event_data/2018-11-13-events.csv..........
Adding /home/workspace/event_data/2018-11-14-events.csv..........
Adding /home/workspace/event_data/2018-11-15-events.csv..........
Adding /ho

In [21]:
# check the number of rows in your csv file

with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [22]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session

session = cluster.connect()

#### Create Keyspace

In [23]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [24]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [31]:
## Functions

def create_table(query)->None:
    """
        create table in cassandra
        
        Parameters:
        session  ---> query session
        query    ---> sql query
    
    """
    try:
        session.execute(query)
    except Exception as e:
        print(e)
    print(f"Table created!")
    
# --------------------------
def show_data(query)->None:
    """
        Function takes a query to see if there is any data inserted.
        
        Parameter:
        query  ---> query statement
        
    """
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

    for row in rows:
        print(row)
    

For table `songs_by_user_session`, the `sessionId` and `itemInsession` was used as a partition key because the queries will filter by this column. Those two columns make up a unique key for the data. Therefore, no clustering key was needed.

In [42]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query1_createTable = """
CREATE TABLE IF NOT EXISTS songs_by_user_session (sessionId INT,
                                                  itemInsession INT,
                                                  artist VARCHAR,
                                                  song VARCHAR,
                                                  length DOUBLE,
                                                  PRIMARY KEY(sessionId,itemInsession))

"""

# create table

create_table(query1_createTable)

Table created!


In [43]:
# We have provided part of the code to set up the CSV file. 

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO songs_by_user_session (sessionId,itemInsession,artist,song,length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [46]:
## Add in the SELECT statement to verify the data was entered into the table

query = "SELECT sessionId,itemInsession,artist,song,length FROM songs_by_user_session WHERE sessionId=338"
show_data(query)

Row(sessionid=338, iteminsession=1, artist='Pixies', song='Build High', length=89.36444)
Row(sessionid=338, iteminsession=2, artist='The Roots / Jack Davey', song='Atonement', length=155.95057)
Row(sessionid=338, iteminsession=3, artist='Mike And The Mechanics', song='A Beggar On A Beach Of Gold', length=275.12118)
Row(sessionid=338, iteminsession=4, artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.3073)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

<img src="images/image_event_datafile_new.jpg">

For table `song_by_artist_user_session`, the `userId` and `sessionId`, were used as a partition key because the queries will filter by those two columns. Column `itemInsession` was used as clustering columns to help make up a unique key.

In [47]:
##  Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2_createTable = """
CREATE TABLE IF NOT EXISTS song_by_artist_user_session (
                                                       userId INT,
                                                       sessionId INT,
                                                       itemInsession INT,
                                                       artist VARCHAR,
                                                       song VARCHAR,
                                                       firstName VARCHAR,
                                                       lastName VARCHAR,
                                                       PRIMARY KEY ((userId, sessionId),itemInsession))



"""
# create table

create_table(query2_createTable)

Table created!


In [48]:
# insert data to table

with open(file,encoding='utf8') as f:
    query='INSERT INTO song_by_artist_user_session(userId,sessionId,itemInsession,artist,song,firstName,lastName) \
           VALUES (%s,%s,%s,%s,%s,%s,%s)'
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(query,(int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

In [49]:
## Add in the SELECT statement to verify the data was entered into the table

test_query = "SELECT firstName,lastName FROM song_by_artist_user_session WHERE userid=10 AND sessionId=182"
show_data(test_query)


Row(firstname='Sylvie', lastname='Cruz')
Row(firstname='Sylvie', lastname='Cruz')
Row(firstname='Sylvie', lastname='Cruz')
Row(firstname='Sylvie', lastname='Cruz')


For table `song_by_user`, the `song` was used as a partition key because the queries will filter by this column. Column `userId` was used as clustering columns to help make up a unique key.

In [50]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3_creatTable = """
CREATE TABLE IF NOT EXISTS song_by_user  (
                                          song VARCHAR,
                                          userId INT,
                                          firstName VARCHAR,
                                          lastName VARCHAR,
                                          PRIMARY KEY(song,userId))

"""

# create table

create_table(query3_creatTable)

Table created!


In [51]:
# insert data to table

with open(file,encoding='utf8') as f:
    query = "INSERT INTO song_by_user (song,userId,firstName,lastName) \
            VALUES (%s,%s,%s,%s)"
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        session.execute(query,(line[9],int(line[10]),line[1],line[4]))

In [52]:
# test the query

test_query ="SELECT firstName,lastName FROM song_by_user WHERE song= 'All Hands Against His Own'"
show_data(test_query)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [53]:
## Drop the table before closing out the sessions

tables= ['songs_by_user_session','song_by_artist_user_session','song_by_user']
for t in tables:
    try:
        session.execute(f'DROP TABLE {t}')
        print(f'Table {t} droped sucessfully!')
    except Exception as e:
        print(f'Drop table {t} failed.')
        print(e)

Table songs_by_user_session droped sucessfully!
Table song_by_artist_user_session droped sucessfully!
Table song_by_user droped sucessfully!


### Close the session and cluster connection¶

In [54]:
session.shutdown()
cluster.shutdown()